In [ ]:
# Imports
import os
import json
import requests
import gradio as gr
from typing import cast
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader

# OpenAI types
from openai.types.chat import ChatCompletionMessageParam

# Markdown output display
from IPython.display import Markdown, display